In [38]:
import pandas as pd

In [39]:
medical_reports = pd.read_csv('reports_copy.csv')

In [40]:
medical_reports.head()

,medical_specialty,report
0,Cardiovascular / Pulmonary,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
1,Cardiovascular / Pulmonary,1. The left ventricular cavity size and wall ...
2,Cardiovascular / Pulmonary,"2-D ECHOCARDIOGRAM,Multiple views of the heart..."
3,Cardiovascular / Pulmonary,"DESCRIPTION:,1. Normal cardiac chambers size...."
4,Cardiovascular / Pulmonary,"2-D STUDY,1. Mild aortic stenosis, widely calc..."


In [41]:
medical_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2179 entries, 0 to 2178
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   medical_specialty  2179 non-null   object
 1   report             2179 non-null   object
dtypes: object(2)
memory usage: 34.2+ KB


In [42]:
medical_reports.dropna(subset=['report'])
medical_reports.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2179 entries, 0 to 2178
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   medical_specialty  2179 non-null   object
 1   report             2179 non-null   object
dtypes: object(2)
memory usage: 34.2+ KB


In [43]:
# Trainning validation 

In [44]:
medical_reports.groupby('medical_specialty').count()

,report
medical_specialty,
Cardiovascular / Pulmonary,371
Gastroenterology,224
Neurology,223
Radiology,273
Surgery,1088


In [45]:
grouped = medical_reports.groupby('medical_specialty').sample(110, random_state=42)
grouped['medical_specialty'].value_counts()

medical_specialty
Cardiovascular / Pulmonary    110
Gastroenterology              110
Neurology                     110
Radiology                     110
Surgery                       110
Name: count, dtype: int64

# From 110 samples of each medical specialty, we will get 5 for validation and 5 for testing

In [46]:
validation = grouped.groupby('medical_specialty').sample(10, random_state=42)

In [47]:
val = validation.groupby('medical_specialty').head(5)
test = validation.groupby('medical_specialty').tail(5)

In [48]:
train = grouped[~grouped.index.isin(val.index)]
len(train)

525

In [49]:
import tiktoken

In [50]:
def number_of_tokens_from_string(string):
    encoding = tiktoken.get_encoding('cl100k_base')
    tokens = len(encoding.encode(string))
    return tokens

In [51]:
number_of_tokens_from_string('hello world how are you')

5

In [52]:
reports_lenghts = train['report'].apply(number_of_tokens_from_string)
reports_lenghts.describe()

count     525.000000
mean      672.158095
std       428.376649
min        14.000000
25%       355.000000
50%       581.000000
75%       900.000000
max      3701.000000
Name: report, dtype: float64

# Data formating

In [53]:
train['medical_specialty'].unique()

array(['Cardiovascular / Pulmonary', 'Gastroenterology', 'Neurology',
       'Radiology', 'Surgery'], dtype=object)

In [54]:
# System Prompt
# USER -> report
# AI -> medical_specialty
system_prompt = 'Given the medical description report classify it into categories: Cardiovascular / Pulmonary, Gastroenterology, Neurology,Radiology, Surgery'
print(system_prompt)

Given the medical description report classify it into categories: Cardiovascular / Pulmonary, Gastroenterology, Neurology,Radiology, Surgery


In [55]:
sample_prompt = {"messages": [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": train['report'].iloc[0]},
    {"role": "assistant", "content": train['medical_specialty'].iloc[0]}
]}
print(sample_prompt)

{'messages': [{'role': 'system', 'content': 'Given the medical description report classify it into categories: Cardiovascular / Pulmonary, Gastroenterology, Neurology,Radiology, Surgery'}, {'role': 'user', 'content': "PREOPERATIVE DIAGNOSES:,1.  Lumbar osteomyelitis.,2.  Need for durable central intravenous access.,POSTOPERATIVE DIAGNOSES:,1.  Lumbar osteomyelitis.,2.  Need for durable central intravenous access.,ANESTHESIA:,  General.,PROCEDURE:,  Placement of left subclavian 4-French Broviac catheter.,INDICATIONS:  ,The patient is a toddler admitted with a limp and back pain, who was eventually found on bone scan and septic workup to have probable osteomyelitis of the lumbar spine at disk areas.  The patient needs prolonged IV antibiotic therapy, but attempt at a PICC line failed.  She has exhausted most of her easy peripheral IV access routes and referral was made to the Pediatric Surgery Service for Broviac placement.  I met with the patient's mom.  With the help of a Spanish inter

In [56]:
def df_to_format(df):
    formatted_data = []
    for index,row in df.iterrows():
        entry = {"messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": row['report']},
            {"role": "assistant", "content": row['medical_specialty']}
        ]}
        formatted_data.append(entry)
    return formatted_data

In [57]:
data = df_to_format(train)
data[10]

{'messages': [{'role': 'system',
   'content': 'Given the medical description report classify it into categories: Cardiovascular / Pulmonary, Gastroenterology, Neurology,Radiology, Surgery'},
  {'role': 'user',
   'content': 'CLINICAL HISTORY: , Patient is a 37-year-old female with a history of colectomy for adenoma.  During her preop evaluation it was noted that she had a lesion on her chest x-ray.  CT scan of the chest confirmed a left lower mass.,SPECIMEN: , Lung, left lower lobe resection.,IMMUNOHISTOCHEMICAL STUDIES:,  Tumor cells show no reactivity with cytokeratin AE1/AE3.  No significant reactivity with CAM5.2 and no reactivity with cytokeratin-20 are seen.  Tumor cells show partial reactivity with cytokeratin-7.  PAS with diastase demonstrates no convincing intracytoplasmic mucin.  No neuroendocrine differentiation is demonstrated with synaptophysin and chromogranin stains.  Tumor cells show cytoplasmic and nuclear reactivity with S100 antibody.  No significant reactivity is d

In [58]:
import json
with open('fine_tuning_data.jsonl', 'w') as f:
    for entry in data:
        f.write(json.dumps(entry))
        f.write('\n')

In [59]:
val_data = df_to_format(val)

In [60]:
with open('validation_data.jsonl', 'w') as f:
    for entry in val_data:
        f.write(json.dumps(entry))
        f.write('\n')

In [85]:
user_prompt = grouped['report'].iloc[1]
user_prompt

NameError: name 'report' is not defined

In [87]:
# Testing the model
from openai import OpenAI
import os
client = OpenAI()

chat_completion = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal::9rqcNd5w", # use custom foundation model
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)
chat_completion.choices[0].message.content.strip()

'Surgery'

In [86]:
# Compare the result with the actual medical specialty

def classify_report(report, model):
    chat_completion = client.chat.completions.create(
        model=model, # use custom foundation model
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": report}
        ]
    )
    return chat_completion

In [88]:
predicted_classes = []
ground_truth_classes = []
for line in grouped.iterrows():
    report, specialty = line[1]['report'], line[1]['medical_specialty']
    ground_truth_classes.append(specialty.strip())
    prediction = classify_report(report, "gpt-3.5-turbo")
    predicted_classes.append(prediction.choices[0].message.content.strip())

KeyboardInterrupt: 

In [94]:
import numpy as np
(np.array(predicted_classes) == np.array(ground_truth_classes)).mean()

ValueError: operands could not be broadcast together with shapes (464,) (465,) 

In [92]:
ground_truth_classes

['Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardiovascular / Pulmonary',
 'Cardio